# 导入环境

In [2]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [3]:
# 读取json数据集文件
df = pd.read_json('../../dataset/huanhuan.json')
ds = Dataset.from_pandas(df)

In [4]:
ds[:3] # 展示前三组数据

{'instruction': ['小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
  '这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。',
  '嬛妹妹，刚刚我去府上请脉，听甄伯母说你来这里进香了。'],
 'input': ['', '', ''],
 'output': ['嘘——都说许愿说破是不灵的。', '你们俩话太多了，我该和温太医要一剂药，好好治治你们。', '出来走走，也是散心。']}

# 处理数据集

In [5]:
model_path = '/root/autodl-tmp/Shanghai_AI_Laboratory/internlm3-8b-instruct'

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)

# 设置tokenizer.pad_token_id 和 tokenizer.eos_token_id,用于token填充和结束符
# 在internlm3-8b-instruct/tokenizer_config.json中, 128131 是 '<|im_end|>' token值
tokenizer.pad_token_id = tokenizer.eos_token_id = 128131  

In [6]:
# 构建messages
messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": '你好呀'},
            {"role": "assistant", "content": '有什么可以帮你的？'}
            ]
# 使用chat_template将messages格式化并打印
print(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))

<s><|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
你好呀<|im_end|>
<|im_start|>assistant
有什么可以帮你的？<|im_end|>
<|im_start|>assistant



In [7]:
system_prompt = '现在你要扮演皇帝身边的女人--甄嬛'

def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    # 构建指令部分的输入
    instruction = tokenizer(
        f"<|im_start|>system\n{system_prompt}<|im_end|>\n" 
        f"<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n"  
        f"<|im_start|>assistant\n",  
        add_special_tokens=False   
    )
    # 构建模型回复部分的输入
    response = tokenizer(
        f"{example['output']}",
        add_special_tokens=False 
    )
    # 拼接指令和回复部分的 input_ids
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    # 拼接指令和回复部分的 attention_mask
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为 EOS token 也需要关注，所以补充为 1
    # 构建标签
    # 对于指令部分，使用 -100 忽略其损失计算；对于回复部分，保留其 input_ids 作为标签
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    # 如果总长度超过最大长度，进行截断
    if len(input_ids) > MAX_LENGTH: 
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [8]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/3729 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3729
})

In [9]:
# 解码输入
print(tokenizer.decode(tokenized_id[1]['input_ids']))

<|im_start|>system
现在你要扮演皇帝身边的女人--甄嬛<|im_end|>
<|im_start|>user
这个温太医啊，也是古怪，谁不知太医不得皇命不能为皇族以外的人请脉诊病，他倒好，十天半月便往咱们府里跑。<|im_end|>
<|im_start|>assistant
你们俩话太多了，我该和温太医要一剂药，好好治治你们。<|im_end|>


In [10]:
# 解码标签, 过滤掉-100
print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"]))))

'你们俩话太多了，我该和温太医要一剂药，好好治治你们。<|im_end|>'

# 创建模型

In [10]:
import torch

model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto",
                                             torch_dtype=torch.bfloat16, trust_remote_code=True)
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

InternLM3ForCausalLM(
  (model): InternLM3Model(
    (embed_tokens): Embedding(128512, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x InternLM3DecoderLayer(
        (self_attn): InternLM3SdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=256, bias=False)
          (v_proj): Linear(in_features=4096, out_features=256, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): InternLM3RotaryEmbedding()
        )
        (mlp): InternLM3MLP(
          (gate_proj): Linear(in_features=4096, out_features=10240, bias=False)
          (up_proj): Linear(in_features=4096, out_features=10240, bias=False)
          (down_proj): Linear(in_features=10240, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): InternLM3RMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): InternLM3RMS

In [11]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [12]:
model.dtype

torch.bfloat16

# lora 

In [13]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["o_proj", "qkv_proj"], # , "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj" # 可以添加更多微调的target_modules
    inference_mode=False, # 训练模式
    r=8,                  # Lora 秩
    lora_alpha=32,        # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1      # Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'o_proj', 'qkv_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [14]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/root/autodl-tmp/Shanghai_AI_Laboratory/internlm3-8b-instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'o_proj', 'qkv_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [15]:
model.print_trainable_parameters()

trainable params: 3,145,728 || all params: 8,807,387,136 || trainable%: 0.0357


# 配置训练参数

In [17]:
import warnings
# 屏蔽讨厌的保存模型权重时UserWarning（Could not find a config file in model_path will assume that the vocabulary was not modified.）
warnings.filterwarnings("ignore") 

output_dir="/root/autodl-tmp/internlm3-8b-instruct_lora_output"

args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, 
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [18]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [28]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,2.859900
20,2.831300
30,2.517200
40,2.904000
50,2.553900
60,2.548200
70,2.657600
80,2.714100
90,2.532200
100,2.687500


/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /root/autodl-tmp/self-llm/models/InternLM3/Shanghai_AI_Laboratory/internlm3-8b-instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /root/autodl-tmp/self-llm/models/InternLM3/Shanghai_AI_Laboratory/internlm3-8b-instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /root/autodl-tmp/self-llm/models/InternLM3/Shanghai_AI_Laboratory/internlm3-8b-instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /root/autodl-tmp/self-llm/models/

TrainOutput(global_step=2796, training_loss=2.470944871206652, metrics={'train_runtime': 1599.2717, 'train_samples_per_second': 6.995, 'train_steps_per_second': 1.748, 'total_flos': 5.238013652747059e+16, 'train_loss': 2.470944871206652, 'epoch': 2.997586484312148})

# 合并加载模型

> 这里推荐大家在**训练结束重启一下notebook, 释放微调占用的GPU显存**, 否则容易出现如下警告"Some parameters are on the meta device because they were offloaded to the cpu."

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

model_path = '/root/autodl-tmp/Shanghai_AI_Laboratory/internlm3-8b-instruct'
lora_path = '/root/autodl-tmp/internlm3-8b-instruct_lora_output/checkpoint-2796' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "你是谁？"
system_prompt = "现在你要扮演皇帝身边的女人--甄嬛"
print("prompt: ", prompt)
print("system_prompt: ", system_prompt)

inputs = tokenizer.apply_chat_template([{"role": "system", "content": system_prompt},
                                        {"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to(model.device)  # 将 inputs 移动到模型所在的设备，确保设备一致性


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print("output: ", tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

prompt:  你是谁？
system_prompt:  现在你要扮演皇帝身边的女人--甄嬛
output:  我是甄嬛，家父是大理寺少卿甄远道。
